# Get data
This notebook fetches annual data using domain.com.au API. Please get your API Keys prior to running this notebook.

Note that the free access only grants certain number of requests a month. Data will have smaller size if the quota is exceeded. In this case, delete those files and try again after a few weeks. If data already exists for some dwelling, no API calls will be made to avoid repetition and save up API call quota. Some postcodes may not have any results (akam there are probably no 4 bedroom house in Melbourne CBD => Null data or error). Any postcodes raising error are stored in `listErrors` list and it may be useful to look at this.


In [1]:
## libraries and packages
import pandas as pd
import numpy as np
import json
from sklearn.linear_model import LinearRegression
import math
import geopandas
import os
import time
import openpyxl
import re
import subprocess


In [2]:
## parameters and export settings
''' 
Data is retrived using nested loops for each element in list Properties and Bedrooms.
For example, following two lists are valid and will fetch 2 and 3 bedroom houses and units for the selected postcodes . 
Properties = ['House', 'Unit']
Bedrooms = ['2', '3'] 
'''


Properties =  ['House'] # valid dwellings in ['House', 'Unit']. Only used House here. Please confirm dwelling types on API documentation.
Bedrooms = ['3'] # number of bedrooms, 
my_periodSize = 'years' # valid periods in ['years', 'quarters']
exportDir = 'rawDumps_Oct2023' # where json data will be dumped. 

## Select relevant postcodes for Melbourne, Victoria
Melbourne_Center_Coord = {'lat': -37.814218,'long': 144.963526} # Center of CBD for calculating distance 
radiusZoneInKM = 60 # select postcodes from THIS radius from Melbourne_Center_Coord


## --
if not os.path.exists(exportDir):
    os.makedirs(exportDir)


In [3]:
## SECRETS and API KEYS
## Paste your keys here
# domain API
client_id = ''
client_secret = ''
MAPBOX_ACCESSTOKEN = ''


## uploading my access keys
exec(open("access_keys.py", "r").read())

In [4]:
## define all functions
def getAccessToken():
    '''Gets Access Token for the session based on pre-defined client_id and client_secret. Sets auth_dict containing access token as global variable.'''
    global auth_dict # stores token
    cmd = 'curl -X POST -u '
    cmd = cmd + '\'' + client_id + ':' + client_secret +'\''
    cmd = cmd + ' -H \'Content-Type: application/x-www-form-urlencoded\'' 
    cmd = cmd + ' -d \'grant_type=client_credentials&scope=api_suburbperformance_read\''
    cmd = cmd + ' \'https://auth.domain.com.au/v1/connect/token\''
    returned_value = subprocess.check_output(cmd, shell = True)
    auth_dict = json.loads(returned_value.decode('utf8'))   

def getSinglePropertyStats_raw(my_property,my_bedrooms, my_postcode, my_suburb,
                               exportDir, 
                               my_state = 'VIC', 
                               my_periodSize='quarters'):
    '''Fetches data on a single query and dumps raw data that into exportDir '''
    fileName = exportDir + '/' + str(my_postcode) + '_' + my_suburb.replace(" ",'-') + '_' + my_state + '_' + my_bedrooms + 'bd_' + my_property + '_'+ my_periodSize + '_raw.json'
    if not os.path.exists(fileName):        
        q = '\'https://api.domain.com.au/v2/suburbPerformanceStatistics'
        q = q + '/' + my_state + '/' + my_suburb + '/' + str(my_postcode) +'?'    
        q = q + 'propertyCategory=' + my_property + '&bedrooms=' + my_bedrooms 
        q = q + '&periodSize='+ my_periodSize +'&startingPeriodRelativeToCurrent=2&totalPeriods=500\''
        url_request = 'curl -X GET -H \'Authorization: Bearer ' + auth_dict['access_token'] + '\''
        url_request = url_request + ' -H \'Content-Type: application/json\' ' 
        url_request = url_request + q
    #     print(url_request)
        q_response = subprocess.check_output(url_request, shell = True).decode('utf8')
    #     q_response_dict = json.loads(q_response)
        f = open(fileName, 'w')
        f.write(q_response)
        f.close()
        print("File written:", fileName)
    else:
        print("Data exists. Skipped:", fileName)
        

def isPointInROI(lat, long, maxDistance = 30):
    '''Checks wheather a point on map falls within a square region of interest.
    length of the ROI is maxDistance x 2, with ref coord of melbourne as center (0,0).
    1 degree on lat and long is roughly 110 km. 
    '''    
    return((abs(lat - Melbourne_Center_Coord['lat']) < (maxDistance/110)) &  (abs(long - Melbourne_Center_Coord['long']) < (maxDistance/110)))




# Begin
## Test authentication

In [ ]:
## authenticate.
## Successful authentication returns a dictionary with 'access_token' : some code, 'expires_in': some time...
getAccessToken()
print(auth_dict)

In [6]:
## Attempt to retrive data for one suburb.
##! if successful, message is 'File written: rawDumps_Oct2023/3181_Kew_VIC_4bd_House_years_raw.json'
##! if file already exists, message is 'Data exists. Skipped: rawDumps_Oct2023/3181_Kew_VIC_4bd_House_years_raw.json'
getSinglePropertyStats_raw('House', '4', '3181', 'Kew', 
                           my_periodSize='years', 
                           exportDir = exportDir)

Data exists. Skipped: rawDumps_Oct2023/3181_Kew_VIC_4bd_House_years_raw.json


## Download data en mass

In [7]:
postcodes = pd.read_csv("Australian_Post_Codes_Lat_Lon.csv")
postcodes = postcodes.query('state == "VIC"' )

## Test whether a postcode is within 
postcodes['inROI'] = postcodes[ ['lat', 'lon'] ].apply(lambda x: isPointInROI(x[0], x[1],radiusZoneInKM) , axis = 1)
postcodes = postcodes[(postcodes.inROI) & (postcodes['postcode'] < 4000)]
len(postcodes.index)

## downloading remining of postcodes
postcodes = postcodes[(postcodes.inROI) & (postcodes['postcode'] >=3000) &  (postcodes['postcode'] <= 3700)]

print("{} potential postcodes found within {} km from the defined centre.".format(str(len(postcodes.index)), str(radiusZoneInKM)))


530 potential postcodes found within 60 km from the defined centre.


In [8]:
# prings all suburbs. maybe useful for quick eyeballing
print(sorted(list(postcodes.suburb)))

['ABBOTSFORD', 'ABERFELDIE', 'AIRPORT WEST', 'ALBANVALE', 'ALBERT PARK', 'ALBION', 'ALPHINGTON', 'ALTONA', 'ALTONA EAST', 'ALTONA GATE', 'ALTONA MEADOWS', 'ALTONA NORTH', 'ARDEER', 'ARMADALE', 'ARMADALE NORTH', 'ARTHURS CREEK', 'ASCOT VALE', 'ASHBOURNE', 'ASHBURTON', 'ASHWOOD', 'ASPENDALE', 'ASPENDALE GARDENS', 'ATTWOOD', 'AUBURN', 'AUBURN SOUTH', 'AVONDALE HEIGHTS', 'BACCHUS MARSH', 'BALACLAVA', 'BALLARAT', 'BALLIANG EAST', 'BALWYN', 'BALWYN EAST', 'BALWYN NORTH', 'BANGHOLME', 'BANYULE', 'BAREENA', 'BARWON HEADS', 'BATMAN', 'BAYSWATER', 'BAYSWATER NORTH', 'BEAUMARIS', 'BEDFORD ROAD', 'BELGRAVE', 'BELGRAVE HEIGHTS', 'BELGRAVE SOUTH', 'BELLARINE', 'BELLFIELD', 'BELVEDERE PARK', 'BEND OF ISLANDS', 'BENDIGO', 'BENNETTSWOOD', 'BENTLEIGH', 'BENTLEIGH EAST', 'BLACK ROCK', 'BLACK ROCK NORTH', 'BLACKBURN', 'BLACKBURN NORTH', 'BLACKBURN SOUTH', 'BOLINDA', 'BONBEACH', 'BORONIA', 'BOX HILL', 'BOX HILL CENTRAL', 'BOX HILL NORTH', 'BOX HILL SOUTH', 'BRAESIDE', 'BRAYBROOK', 'BRAYBROOK NORTH', 'BRENT

In [9]:
## final step. Download data. Be mindful of number of requests allowed by API as well as frequncy of requests sent
## request will be skipped if data already exists in the export dir.

listErrors = [] # catch problematic postcodes
# completed: ['House', '2'], ['Unit', '2'], ['Unit','1'], ['Unit','3'], ['House', '1'], ['House', '3'], ['House', '4']
if(not os.path.exists(exportDir)):
    os.makedirs(exportDir)
    
for my_postcode, my_suburb, my_state in zip(postcodes.postcode, postcodes.suburb, postcodes.state):
    for my_property in Properties:
        for my_bedrooms in Bedrooms:
            try:
                getSinglePropertyStats_raw(my_property,my_bedrooms, my_postcode, my_suburb,
                           exportDir, 
                           my_state = list(postcodes.state.unique())[0], 
                           my_periodSize = my_periodSize)
                time.sleep(0.1)
            except:
                temp_d = {'postcode': my_postcode, 
                                      'suburb' : my_suburb,
                                      'state': my_state,
                                      'propertyCategory' : my_property,
                                     'bedrooms' : my_bedrooms,
                                     'error_code': 500}
                listErrors.append(temp_d)
#                 print(temp_d)
print("-- Done --")              

Data exists. Skipped: rawDumps_Oct2023/3000_MELBOURNE_VIC_3bd_House_years_raw.json


curl: (3) URL using bad/illegal format or missing URL
curl: (3) URL using bad/illegal format or missing URL


Data exists. Skipped: rawDumps_Oct2023/3004_MELBOURNE_VIC_3bd_House_years_raw.json
Data exists. Skipped: rawDumps_Oct2023/3006_SOUTHBANK_VIC_3bd_House_years_raw.json


curl: (3) URL using bad/illegal format or missing URL


Data exists. Skipped: rawDumps_Oct2023/3008_DOCKLANDS_VIC_3bd_House_years_raw.json


...

curl: (3) URL using bad/illegal format or missing URL
curl: (3) URL using bad/illegal format or missing URL


Data exists. Skipped: rawDumps_Oct2023/3502_MILDURA_VIC_3bd_House_years_raw.json
Data exists. Skipped: rawDumps_Oct2023/3552_BENDIGO_VIC_3bd_House_years_raw.json
Data exists. Skipped: rawDumps_Oct2023/3570_SUMMERFIELD_VIC_3bd_House_years_raw.json
Data exists. Skipped: rawDumps_Oct2023/3619_KYABRAM_VIC_3bd_House_years_raw.json
Data exists. Skipped: rawDumps_Oct2023/3658_CLONBINANE_VIC_3bd_House_years_raw.json
Data exists. Skipped: rawDumps_Oct2023/3658_FLOWERDALE_VIC_3bd_House_years_raw.json
Data exists. Skipped: rawDumps_Oct2023/3658_HAZELDENE_VIC_3bd_House_years_raw.json


curl: (3) URL using bad/illegal format or missing URL


Data exists. Skipped: rawDumps_Oct2023/3689_WODONGA_VIC_3bd_House_years_raw.json
-- Done --


curl: (3) URL using bad/illegal format or missing URL


In [10]:
## Inspect errors
pd.DataFrame(listErrors)

,postcode,suburb,state,propertyCategory,bedrooms,error_code
0,3002,EAST MELBOURNE,VIC,House,3,500
1,3003,WEST MELBOURNE,VIC,House,3,500
2,3005,WORLD TRADE CENTRE,VIC,House,3,500
3,3010,UNIVERSITY OF MELBOURNE,VIC,House,3,500
4,3011,SEDDON WEST,VIC,House,3,500
...,...,...,...,...,...,...
256,3442,HANGING ROCK,VIC,House,3,500
257,3442,WOODEND NORTH,VIC,House,3,500
258,3444,KYNETON SOUTH,VIC,House,3,500
259,3658,WATERFORD PARK,VIC,House,3,500
